In [1]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import ttest_ind, norm, chi2_contingency, f_oneway

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

## ETL

En este caso trabajas en una empresa de venta al por menor de productos italianos y debes realizar la limpieza, transformación e integración de datos de ventas, productos y clientes para su análisis.

1. Lectura de la Información:

    - Leer los archivos CSV (ventas.csv, productos.csv, clientes.csv).

    - Explorar los conjuntos de datos para comprender su estructura, columnas, tipos de datos, etc.

2. Transformación de Datos:

    - Limpiar los datos: manejar valores nulos, eliminar duplicados si los hay, corregir errores tipográficos, etc.

    - Realizar la integración de datos: unir los conjuntos de datos apropiados para obtener una tabla única que contenga información de ventas junto con detalles de productos y clientes.

    - Aplicar transformaciones relevantes según sea necesario: por ejemplo, convertir tipos de datos, renombrar columnas, crear nuevas características derivadas, etc.

In [8]:
with open("files/productos.csv", 'r') as file:

    lines = file.readlines()

In [13]:
print(lines[9])
print(lines[10])


I9,Lasagna Bolognesa,Platos Preparados,7.49,Italia,Clásica lasaña con salsa boloñesa, bechamel y queso gratinado, lista para hornear.

J10,Arroz Arborio,Productos Secos,4.89,Italia,Arroz de grano corto perfecto para risottos italianos, absorbe los sabores de forma exquisita.



In [6]:
customer = pd.read_csv("files/clientes.csv", index_col = 0)
sales = customer = pd.read_csv("files/ventas.csv", index_col = 0)
